
## NTRU 2015
A implementação de NTRU versão de 2015 descrita no paper "**NTRU and Lattice-Based Crypto: Past, Present, and Future**" de *Joseph H. Silverman*, Brown University apresentado no encontro The Mathematics of Post-Quantum Cryptography, DIMACS Center, Rutgers University em 12–16 de Janeiro de 2015.

## Parâmetros
Vamos usar
+ $d \ge 6$ --
  parâmetro de segurança
+ $N$ --
  um primo maior ou igual  2^d
  
+ $p$ --
  um pequeno primo  3,  5 ou  7
 
+ $q$ --
  um primo da ordem de grandeza de  $p*\mbox{poly}(N)$
 

In [1]:
d = 10
N = next_prime(1 << d)

p = 3
q = next_prime(p*N)

print (q, p, N)

3109 3 1031


### anéis de polinómios

In [14]:
Z.<x>  = ZZ[]        # polinómios de coeficientes inteiros
Q.<x>  = PolynomialRing(GF(q),name='x').quotient(x^N-1)

### geração aleatória, arredondamento módulo $q$ e compração módulo $p$

In [3]:
def vec():
    return  [choice([-1,0,1]) for k in range(N)]

# arredondamento módulo 'q'
def qrnd(f):    # argumento em 'Q'
    qq = (q-1)//2
    ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

# arredondamento módulo 'p'
def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p        
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

### classe NTRU

In [4]:
class NTRU(object):
    def __init__(self):
        # calcular um 'f' invertível
        f = Q(0)
        while not f.is_unit():
            F = Q(vec())
            f = 1 + p*F
        # gerar as chaves
        G = Q(vec())
        g = p*G
        self.f = qrnd(f)
        self.h = qrnd(f^(-1) * g)
        
    def encrypt(self,m):
        r = Q(vec()) 
        return qrnd(r*Q(self.h) + Q(m))

    def decrypt(self,e):
        a = Q(e)*Q(self.f)
        return prnd(qrnd(a))


### Teste

In [5]:
# Uma instância NTRU
K = NTRU() 
# Uma mensagem aleatória
m = vec()
#print(m)
# Cifrar
e = K.encrypt(m)  
# Decifrar e Verificar
m == K.decrypt(e)


True

## Reticulado $L(h)$

In [6]:
# Construção da matriz geradora por blocos

import sage.modules.free_module_integer as fmi
# http://doc.sagemath.org/html/en/reference/modules/sage/modules/free_module_integer.html

class Lat(NTRU):
    def __init__(self):
        super(Lat,self).__init__()
        B1 = identity_matrix(ZZ,N); Bq = q*B1; B0 = matrix(ZZ,N,N,[0]*(N^2))
        # rodar um vetor
        h = self.h
        H = [h]
        for k in range(N-1):
            h = [h[-1]] + h[:-1]   # shift right rotate
            H = H + [h]
        H = matrix(ZZ,N,N,H)

        self.L = fmi.IntegerLattice(block_matrix([[Bq,B0],[H,B1]]))   
    
    def short_basis(self):
        sb = []
        for i in range(N):
            u = self.L.reduced_basis[i].list()
            u[0] = u[0] - 1
            sb += [map(int,u)]
        return sb
            
   


In [7]:
l = Lat()
f = l.f
print f
sb =  l.short_basis()
print sb

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(f)? (<ipython-input-7-488313ba6949>, line 3)

In [ ]:
import pickle

fd = open("base","wb")
pickle.dump(sb,fd)
fd.close()